In [60]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

if not firebase_admin._apps:
    cred = credentials.Certificate("../jsonFile/service-account.json")
    firebase_admin.initialize_app(cred)
    
db = firestore.client()

In [61]:
import requests
from bs4 import BeautifulSoup

label='rf_systems'
url = 'https://scholar.google.com.tw/citations?view_op=search_authors&hl=zh-TW&mauthors=label:'+ label
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'
}
r = requests.get(url, headers=headers)
soup = BeautifulSoup(r.text)

In [62]:
searchPage = True
count = 1

temp = []
user = {}
page = int('0')
  
while searchPage == True:
    for i in soup.find_all('div', class_ = 'gsc_1usr'):      
        #tag = []

        id = i.find('a')['href'].split('user=')[1]
        temp.append(id)
        #temp['label'] = tag
        
    user['userID'] = temp
    try:
        #split nextpage after_author
        afterAuthor = soup.find('div', id = 'gsc_authors_bottom_pag').find('button', class_ = 'gs_btnPR gs_in_ib gs_btn_half gs_btn_lsb gs_btn_srt gsc_pgn_pnx')['onclick'].split('author\\x3d')[1].split('\\x26')[0]
        page = str(int(page) + 10) 
        
        url = 'https://scholar.google.com.tw/citations?view_op=search_authors&hl=zh-TW&mauthors=label:'+label+'&after_author='+afterAuthor+'&astart='+page
        r = requests.get(url, headers=headers)
        soup = BeautifulSoup(r.text)
        
        count = count +1
        print(count)
    except KeyError as err:        
        searchPage == False
        break

            


AttributeError: 'NoneType' object has no attribute 'find'

In [4]:
import json
def jsontransfer(items):
    data = json.dumps(items)
    jsonStr = json.loads(data) 
    return jsonStr

In [38]:
def update_labelinfo(items, label):
    print(label)
    ref = db.collection(u'Label-Domain').document(label)
    ref.set(items)

In [21]:
updateLabel(user, label)

In [5]:
import requests
from bs4 import BeautifulSoup
import datetime
def currentTime():
    now = datetime.datetime.now()
    currentTime = now.strftime("%Y-%m-%d %H:%M:%S")
    return currentTime

In [44]:
def get_labelIDlist(label):

    url = 'https://scholar.google.com.tw/citations?view_op=search_authors&hl=zh-TW&mauthors=label:' + label
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'
    }
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text)

    searchPage = True
    tempList = []
    Label = {}
    page = int('0')

    while searchPage == True:
        for i in soup.find_all('div', class_='gsc_1usr'):
            #tag = []

            id = i.find('a')['href'].split('user=')[1]
            tempList.append(id)
            #temp['label'] = tag

        Label['userID'] = tempList
        try:
            # split nextpage after_author
            afterAuthor = soup.find('div', id='gsc_authors_bottom_pag').find(
                'button', class_='gs_btnPR gs_in_ib gs_btn_half gs_btn_lsb gs_btn_srt gsc_pgn_pnx')['onclick'].split('author\\x3d')[1].split('\\x26')[0]
            page = str(int(page) + 10)

            url = 'https://scholar.google.com.tw/citations?view_op=search_authors&hl=zh-TW&mauthors=label:' + \
                label+'&after_author='+afterAuthor+'&astart='+page
            r = requests.get(url, headers=headers)
            soup = BeautifulSoup(r.text)
            print(page)
        except :
            searchPage == False
            break
    Label['updateTime'] = currentTime()
    return Label

In [48]:
 t=get_LabelIDList('5G Wireless Communications')
print(t)

{'userID': [], 'updateTime': '2021-10-17 21:30:01'}


In [22]:
print(type(t))

<class 'dict'>


In [47]:
updateLabel(t, '5G Wireless Communications')

In [40]:
def labelinfoformat(data):
    rewriteData = data
    if(len(data['userID']) == 0):
        print(len(data['userID']))
        return False
    # check updateTime format
    if(data['updateTime'] == '' or data['updateTime'] == None):
        rewriteData['personalData']['updateTime'] = currentTime()
        print(2)

    try:
        datetime.datetime.strptime(
            data['updateTime'], "%Y-%m-%d %H:%M:%S")
        print(1)
    except:
        rewriteData['updateTime'] = currentTime()
        print(3)
    return rewriteData

In [41]:

def LabelCrawl():
    print('label start')
    label = get_lastupdatelabel()  # limit
    labellist = get_labelIDlist(label)
    print(labellist)
    check_labelformat = labelinfoformat(labellist)

    # label list 為空或格式錯誤時回傳False,格式錯誤修正後回傳rewriteInfo
    if(check_labelformat != False):
        update_labelinfo(labellist, label)
        print('label 1done')
    else:
        rewrite_labelinfo = check_labelformat
        update_labelinfo(rewrite_labelinfo, label)
        print('label done')

In [62]:
p= {'userID': ['NjciM8IAAAAJ', '_FsNXuIAAAAJ', '8UU9ZfYAAAAJ', 'X_arEpUAAAAJ', 'ptBfinUAAAAJ', 'ZyX78GgAAAAJ', 'N-SEWfcAAAAJ', 'lmOFlZIAAAAJ', '683EoygAAAAJ', 'w-3Ew7wAAAAJ', 'KhHjFWMAAAAJ', 'E57IHlUAAAAJ', 'uyoWO5gAAAAJ', 'qXf3SpwAAAAJ', 'To17VOkAAAAJ', 'FhcloGAAAAAJ', '5eWXa0YAAAAJ', 'eD_xT7UAAAAJ', 'd8QkDAwAAAAJ', 'CAoTUEgAAAAJ', 'rGZmioMAAAAJ', 'heg5lugAAAAJ', 'IVlZ1wsAAAAJ', 'j-NMdXcAAAAJ', 'Qjqh8HUAAAAJ', 'biE0BGcAAAAJ', 'm5lXn6oAAAAJ', '-NqEMJcAAAAJ', 'Af66FWsAAAAJ', 'eRTlT2wAAAAJ', 'yHtxQcQAAAAJ', 'DoPZHWcAAAAJ', 'A3z3s24AAAAJ', 'OCbSI08AAAAJ', 'CLJ4pmEAAAAJ', '4lodKcsAAAAJ', '9CIaAwcAAAAJ', '-7NOZu0AAAAJ', '_BFAbiYAAAAJ', '5pxbliIAAAAJ', 'pcgCwM0AAAAJ', 'CS6GfDYAAAAJ', 'T1-O8JEAAAAJ', '0HMabSEAAAAJ', 'c2UuvMUAAAAJ', 'HZsS6BUAAAAJ', 'wlZwsdQAAAAJ', 'I6obUmQAAAAJ', 'AAQh4bAAAAAJ', 'YzZ6WPgAAAAJ', 'vcj74QYAAAAJ', 'pkTofqsAAAAJ', 'm5CfhiMAAAAJ', '5a-LqosAAAAJ', 'WKP9choAAAAJ', 'opqKtpgAAAAJ', 'QOOVr8AAAAAJ', '3eV7wJIAAAAJ', 'b7zC8YkAAAAJ', 'lU2C_bwAAAAJ', 'Ff8XTpMAAAAJ', 'jCY3UIcAAAAJ', 'nPyakUUAAAAJ', 'pkISiIsAAAAJ', 'GmkF9BsAAAAJ', 'B747ynEAAAAJ', 'PF6ZKJkAAAAJ', 'jYb3FjYAAAAJ', 'xbKwlO4AAAAJ', 'tcuhPQYAAAAJ', '75VB9hAAAAAJ', 'rm7HkcMAAAAJ'], 'label': 'rf_systems', 'updateTime': ''}

In [42]:
def get_lastupdatelabel():
    query = db.collection(u'Label-Domain').where(u'updateTime', '==', None).limit(1)
    results = query.stream()
    for r in results:
        label = r.id
    return label

In [45]:
LabelCrawl()

label start
{'userID': [], 'updateTime': '2021-10-17 21:28:13'}
0
5G Wireless Communications
label done
